In [3]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import joblib

In [4]:
OUT_DIR = './titanic_output'
os.makedirs(OUT_DIR, exist_ok=True)

CSV_PATH = '/content/data.csv'
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"{CSV_PATH} not found in current directory. Download from Kaggle and place it here.")

df = pd.read_csv(CSV_PATH)
print('Loaded dataset shape:', df.shape)

Loaded dataset shape: (891, 12)


In [5]:
# columns useful for prediction
keep_cols = ['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
df = df[keep_cols].copy()

print(df.head())
print('\nMissing values per column:\n', df.isnull().sum())

# Handle missing values
num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

df['Age'] = num_imputer.fit_transform(df[['Age']])
df['Fare'] = num_imputer.fit_transform(df[['Fare']])
df['Embarked'] = cat_imputer.fit_transform(df[['Embarked']]).ravel()

# Encode categorical variables
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1}).astype(int)
emb_ohe = pd.get_dummies(df['Embarked'], prefix='Emb')
df = pd.concat([df.drop(columns=['Embarked']), emb_ohe], axis=1)

# Features and target
target = 'Survived'
feature_cols = [c for c in df.columns if c != target]

# Standardize numerical features: Age, Fare, SibSp, Parch, Pclass
num_cols = ['Age','Fare','SibSp','Parch','Pclass']
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])


   Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0         0       3    male  22.0      1      0   7.2500        S
1         1       1  female  38.0      1      0  71.2833        C
2         1       3  female  26.0      0      0   7.9250        S
3         1       1  female  35.0      1      0  53.1000        S
4         0       3    male  35.0      0      0   8.0500        S

Missing values per column:
 Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64


In [6]:
# Save prepared dataset
prepared_path = os.path.join(OUT_DIR,'titanic_prepared.csv')
df.to_csv(prepared_path, index=False)
print('Saved prepared dataset to', prepared_path)

report_lines = []
report_lines.append('# Titanic Dataset - EDA & Preprocessing Report')
report_lines.append('\n## Part 1: Data Understanding & Preprocessing')
report_lines.append('\nColumns used: ' + ', '.join(feature_cols) + '\n')

# Summary stats for numeric columns
stats = df[num_cols].agg(['mean','median','std']).T
report_lines.append('\n### Numeric feature statistics')
report_lines.append(stats.to_markdown())

# Outlier detection using IQR for Age and Fare
outlier_dict = {}
for col in ['Age','Fare']:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    count = ((df[col] < lower) | (df[col] > upper)).sum()
    outlier_dict[col] = {'lower':float(lower),'upper':float(upper),'num_outliers':int(count)}

report_lines.append('\n### Outlier info (IQR method)')
report_lines.append(pd.DataFrame(outlier_dict).T.to_markdown())

Saved prepared dataset to ./titanic_output/titanic_prepared.csv


In [7]:
# Visualizations
# 1) Histogram: Age
plt.figure(figsize=(7,4))
plt.hist(df['Age'], bins=30)
plt.title('Histogram: Age (standardized)')
plt.xlabel('Age (standardized)')
plt.ylabel('Count')
plt.tight_layout()
hist_path = os.path.join(OUT_DIR,'hist_age.png')
plt.savefig(hist_path)
plt.close()

# 2) Scatter: Age vs Fare colored by Survived
plt.figure(figsize=(7,5))
colors = df['Survived'].map({0:'red',1:'green'})
plt.scatter(df['Age'], df['Fare'], c=colors, alpha=0.6)
plt.title('Scatter: Age vs Fare (color=Survived)')
plt.xlabel('Age (standardized)')
plt.ylabel('Fare (standardized)')
plt.tight_layout()
scatter_path = os.path.join(OUT_DIR,'scatter_age_fare.png')
plt.savefig(scatter_path)
plt.close()

# 3) Correlation heatmap
corr = df.corr()
plt.figure(figsize=(8,6))
plt.imshow(corr, interpolation='nearest', cmap='coolwarm')
plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha='right')
plt.yticks(range(len(corr.columns)), corr.columns)
plt.colorbar()
plt.title('Correlation matrix')
plt.tight_layout()
heatmap_path = os.path.join(OUT_DIR,'corr_heatmap.png')
plt.savefig(heatmap_path)
plt.close()

report_lines.append('\nSaved visualizations:')
report_lines.append(f'- {hist_path}')
report_lines.append(f'- {scatter_path}')
report_lines.append(f'- {heatmap_path}\n')


In [8]:


report_lines.append('\n## Part 3: Model Building')
X = df[feature_cols]
y = df[target]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

# Model 1: Logistic Regression
lr = LogisticRegression(max_iter=500)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

# Model 2: Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

In [9]:

# Evaluation function
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def eval_metrics(y_true, y_pred):
    return {
        'accuracy': float(accuracy_score(y_true, y_pred)),
        'precision': float(precision_score(y_true, y_pred, zero_division=0)),
        'recall': float(recall_score(y_true, y_pred, zero_division=0)),
        'f1': float(f1_score(y_true, y_pred, zero_division=0)),
        'confusion_matrix': confusion_matrix(y_true, y_pred).tolist()
    }

metrics_lr = eval_metrics(y_test, y_pred_lr)
metrics_rf = eval_metrics(y_test, y_pred_rf)

report_lines.append('\n### Baseline performance (before tuning)')
report_lines.append('\n**Logistic Regression:**')
report_lines.append(str(metrics_lr))
report_lines.append('\n**Random Forest:**')
report_lines.append(str(metrics_rf))

#  confusion matrix image for RF
plt.figure(figsize=(5,4))
plt.imshow(confusion_matrix(y_test,y_pred_rf))
plt.title('Confusion Matrix - Random Forest (before tuning)')
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('Actual')
cm_before_path = os.path.join(OUT_DIR,'cm_rf_before.png')
plt.savefig(cm_before_path)
plt.close()

report_lines.append('\n## Part 4: Optimization (GridSearchCV)')
param_grid = {
    'n_estimators': [50,100,200],
    'max_depth': [None, 5, 8],
    'min_samples_split': [2,4]
}
gs = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=4, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)

best_rf = gs.best_estimator_
print('Best params:', gs.best_params_)

Best params: {'max_depth': None, 'min_samples_split': 4, 'n_estimators': 50}


In [11]:
# Evaluate tuned model
y_pred_best = best_rf.predict(X_test)
metrics_best = eval_metrics(y_test, y_pred_best)
report_lines.append('\nBest params: ' + str(gs.best_params_))
report_lines.append('\nPerformance after tuning:')
report_lines.append(str(metrics_best))

# confusion matrix after tuning
plt.figure(figsize=(5,4))
plt.imshow(confusion_matrix(y_test,y_pred_best))
plt.title('Confusion Matrix - Random Forest (after tuning)')
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('Actual')
cm_after_path = os.path.join(OUT_DIR,'cm_rf_after.png')
plt.savefig(cm_after_path)
plt.close()

# Compare improvements
improvements = {
    'accuracy_before': metrics_rf['accuracy'],
    'accuracy_after': metrics_best['accuracy'],
    'f1_before': metrics_rf['f1'],
    'f1_after': metrics_best['f1']
}
report_lines.append('\nImprovements (Random Forest):')
report_lines.append(str(improvements))

# Save tuned model
model_path = os.path.join(OUT_DIR,'best_random_forest.joblib')
joblib.dump(best_rf, model_path)
report_lines.append('\nSaved tuned model to: ' + model_path)

In [16]:
import pandas as pd

df = pd.read_csv('/content/data.csv')

keep_cols = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Survived']
df = df[keep_cols].copy()

df['Age'].fillna(df['Age'].median(), inplace=True)
df['Fare'].fillna(df['Fare'].median(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

df['Sex'] = df['Sex'].map({'male':0, 'female':1})


/tmp/ipython-input-1382533630.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
/tmp/ipython-input-1382533630.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [17]:
num_cols = ['Pclass','Sex','Age','SibSp','Parch','Fare']
mean_vals = df[num_cols].mean().values
std_vals = df[num_cols].std().values


In [18]:
import numpy as np

def scale_numeric(new_input):
    return (np.array(new_input) - mean_vals) / std_vals


In [20]:
import joblib

# Load trained model
model = joblib.load('/content/titanic_output/best_random_forest.joblib')

# Pclass=3, Sex=female, Age=29, SibSp=0, Parch=0, Fare=15.0, Embarked=S
num_input = scale_numeric([3, 1, 29, 0, 0, 15.0])

# One-hot encode Embarked
emb_dict = {'C':[1,0,0], 'Q':[0,1,0], 'S':[0,0,1]}
emb_input = np.array(emb_dict['S'])

# Combine numeric + categorical
X_new = np.hstack([num_input, emb_input]).reshape(1,-1)

# Predict
pred = model.predict(X_new)
print('Prediction:', 'Survived' if pred[0]==1 else 'Did not survive')


Prediction: Survived


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
